In [1]:
import pandas as pd # type: ignore
from datetime import datetime # type: ignore
import glob
# https://github.com/greysonchung/Battery-Optimisation/blob/main/algorithms/battery_optimise.ipynb

#### Podatki

- df_eles_koledar
	- df_eles
	- df_prazniki
	- df_weather

- df_entsoe

- df_all_prices
	- ida
		- ida1
		- ida2
		- sipx
	- df_prices_volumes
		- df_prices
		- df_volumes

### df_eles_koledar

##### df_eles

In [2]:
df_eles = pd.read_csv('data/other/ELES.csv', sep=',', index_col=0)
df_eles.rename(columns={'Unnamed: 0.1': 'datum', 'PREDVIDEN PREVZEM': 'predviden_prevzem', 'DEJANSKI PREVZEM': 'dejanski_prevzem'}, inplace=True)
df_eles['datum'] = pd.to_datetime(df_eles.index)
df_eles.drop(columns=['hour'], inplace=True)
df_eles

,PREDVIDENA PROIZVODNJA,DEJANSKA PROIZVODNJA,predviden_prevzem,dejanski_prevzem,datum
2022-01-01 00:00:00,965.0,968.0,989.0,1044.0,2022-01-01 00:00:00
2022-01-01 01:00:00,959.0,949.0,953.0,1018.0,2022-01-01 01:00:00
2022-01-01 02:00:00,960.0,948.0,906.0,975.0,2022-01-01 02:00:00
2022-01-01 03:00:00,959.0,947.0,874.0,944.0,2022-01-01 03:00:00
2022-01-01 04:00:00,960.0,946.0,870.0,936.0,2022-01-01 04:00:00
...,...,...,...,...,...
2024-11-18 19:00:00,1845.0,NaN,1788.0,NaN,2024-11-18 19:00:00
2024-11-18 20:00:00,1658.0,NaN,1714.0,NaN,2024-11-18 20:00:00
2024-11-18 21:00:00,1323.0,NaN,1602.0,NaN,2024-11-18 21:00:00
2024-11-18 22:00:00,1160.0,NaN,1498.0,NaN,2024-11-18 22:00:00


##### df_prazniki

In [3]:
from prophet.make_holidays import make_holidays_df # type: ignore

df_prazniki = make_holidays_df(
    year_list=[2022 + i for i in range(3)], country='SI'
)
df_prazniki = df_prazniki.rename(columns={'ds': 'datum'})
df_prazniki.sort_values('datum', inplace=True)
df_prazniki['datum'] = pd.to_datetime(df_prazniki['datum'])

# Generate a new DataFrame with all dates between 2022-01-01 and 2024-10-23
df_all_dates = pd.DataFrame({'datum': pd.date_range('2022-01-01', '2024-12-31')})

# Merge the new DataFrame with df_prazniki, filling in missing values with 0
df_prazniki = pd.merge(df_all_dates, df_prazniki, on='datum', how='outer').fillna(0)

df_prazniki.tail()

d:\Projects\Programming\Gen-I Trading Challenge\Task 2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,datum,holiday
1091,2024-12-27,0
1092,2024-12-28,0
1093,2024-12-29,0
1094,2024-12-30,0
1095,2024-12-31,0


##### df_weather

In [4]:
df_forecast = pd.read_csv('data/weather_fore.csv', sep=',')
df_forecast = df_forecast.iloc[:, [0, 1, 9, 17]]
df_forecast['datum'] = pd.to_datetime(df_forecast['date']).apply(lambda t: t.replace(tzinfo=None))
df_forecast.drop(columns=['date'], inplace=True)
df_forecast

,temperature_2m,cloud_cover,direct_normal_irradiance,datum
0,7.124,94.0,0.0,2024-11-02 23:00:00
1,8.074,99.0,0.0,2024-11-03 00:00:00
2,8.274,100.0,0.0,2024-11-03 01:00:00
3,8.524,100.0,0.0,2024-11-03 02:00:00
4,8.674,100.0,0.0,2024-11-03 03:00:00
...,...,...,...,...
499,3.415,86.0,0.0,2024-11-23 18:00:00
500,4.065,90.0,0.0,2024-11-23 19:00:00
501,4.665,93.0,0.0,2024-11-23 20:00:00
502,5.165,97.0,0.0,2024-11-23 21:00:00


In [5]:
folder_path = 'data/weather'
df_weather = pd.concat([pd.read_csv(file) for file in glob.glob(folder_path + '/*.csv')], ignore_index=True)

df_weather = df_weather.iloc[:, [0,1,2,3]]
df_weather['datum'] = pd.to_datetime(df_weather['date']).apply(lambda t: t.replace(tzinfo=None))
df_weather.drop(columns=['date'], inplace=True)
df_weather

df_weather = pd.concat([df_weather, df_forecast])
df_weather

,temperature_2m,cloud_cover,direct_normal_irradiance,datum
0,5.130,0.0,0.0,2021-12-31 23:00:00
1,2.780,8.0,0.0,2022-01-01 00:00:00
2,3.580,14.0,0.0,2022-01-01 01:00:00
3,3.630,2.0,0.0,2022-01-01 02:00:00
4,3.980,23.0,0.0,2022-01-01 03:00:00
...,...,...,...,...
499,3.415,86.0,0.0,2024-11-23 18:00:00
500,4.065,90.0,0.0,2024-11-23 19:00:00
501,4.665,93.0,0.0,2024-11-23 20:00:00
502,5.165,97.0,0.0,2024-11-23 21:00:00


##### Merge df_eles_koledar

In [6]:
df_temp = pd.merge(df_eles, df_prazniki, on='datum', how='left')
df_eles_koledar = pd.merge(df_temp, df_weather, on='datum', how='right')
df_eles_koledar

,PREDVIDENA PROIZVODNJA,DEJANSKA PROIZVODNJA,predviden_prevzem,dejanski_prevzem,datum,holiday,temperature_2m,cloud_cover,direct_normal_irradiance
0,NaN,NaN,NaN,NaN,2021-12-31 23:00:00,NaN,5.130,0.0,0.0
1,965.0,968.0,989.0,1044.0,2022-01-01 00:00:00,New Year's Day,2.780,8.0,0.0
2,959.0,949.0,953.0,1018.0,2022-01-01 01:00:00,NaN,3.580,14.0,0.0
3,960.0,948.0,906.0,975.0,2022-01-01 02:00:00,NaN,3.630,2.0,0.0
4,959.0,947.0,874.0,944.0,2022-01-01 03:00:00,NaN,3.980,23.0,0.0
...,...,...,...,...,...,...,...,...,...
25771,NaN,NaN,NaN,NaN,2024-11-23 18:00:00,NaN,3.415,86.0,0.0
25772,NaN,NaN,NaN,NaN,2024-11-23 19:00:00,NaN,4.065,90.0,0.0
25773,NaN,NaN,NaN,NaN,2024-11-23 20:00:00,NaN,4.665,93.0,0.0
25774,NaN,NaN,NaN,NaN,2024-11-23 21:00:00,NaN,5.165,97.0,0.0


### df_entsoe

##### df_total_load

In [7]:
folder_path = 'data/entsoe/total_load_day_ahead'
df_total_load = pd.concat([pd.read_csv(file) for file in glob.glob(folder_path + '/*.csv')], ignore_index=True)

# Concatenate all the DataFrames together
df_total_load['datum'] = pd.to_datetime(df_total_load['Time (CET/CEST)'].str.split(pat=' - ', expand=True)[0],
										format='%d.%m.%Y %H:%M')
df_total_load = df_total_load.drop(columns=['Time (CET/CEST)'])
df_total_load

,Day-ahead Total Load Forecast [MW] - BZN|SI,Actual Total Load [MW] - BZN|SI,datum
0,1089.0,1150.0,2022-01-01 00:00:00
1,1056.0,1127.0,2022-01-01 01:00:00
2,1008.0,1083.0,2022-01-01 02:00:00
3,976.0,1053.0,2022-01-01 03:00:00
4,973.0,1047.0,2022-01-01 04:00:00
...,...,...,...
26302,-,-,2024-12-31 19:00:00
26303,-,-,2024-12-31 20:00:00
26304,-,-,2024-12-31 21:00:00
26305,-,-,2024-12-31 22:00:00


##### df_forecast_solar

In [8]:
folder_path = 'data/entsoe/generation_forecast_solar'
df_forecast_solar = pd.concat([pd.read_csv(file) for file in glob.glob(folder_path + '/*.csv')], ignore_index=True)
df_forecast_solar['datum'] = pd.to_datetime(df_forecast_solar['MTU (CET/CEST)'].str.split(pat=' - ', expand=True)[0],
										format='%d.%m.%Y %H:%M')
df_forecast_solar = df_forecast_solar.iloc[:, [1, -1]]
df_forecast_solar

,Generation - Solar [MW] Day Ahead/ Slovenia (SI),datum
0,0.0,2022-01-01 00:00:00
1,0.0,2022-01-01 01:00:00
2,0.0,2022-01-01 02:00:00
3,0.0,2022-01-01 03:00:00
4,0.0,2022-01-01 04:00:00
...,...,...
26302,NaN,2024-12-31 19:00:00
26303,NaN,2024-12-31 20:00:00
26304,NaN,2024-12-31 21:00:00
26305,NaN,2024-12-31 22:00:00


##### df_entsoe - merge

In [9]:
df_entsoe = pd.merge(df_total_load, df_forecast_solar, on='datum', how='left')
df_entsoe

,Day-ahead Total Load Forecast [MW] - BZN|SI,Actual Total Load [MW] - BZN|SI,datum,Generation - Solar [MW] Day Ahead/ Slovenia (SI)
0,1089.0,1150.0,2022-01-01 00:00:00,0.0
1,1056.0,1127.0,2022-01-01 01:00:00,0.0
2,1008.0,1083.0,2022-01-01 02:00:00,0.0
3,976.0,1053.0,2022-01-01 03:00:00,0.0
4,973.0,1047.0,2022-01-01 04:00:00,0.0
...,...,...,...,...
26308,-,-,2024-12-31 19:00:00,NaN
26309,-,-,2024-12-31 20:00:00,NaN
26310,-,-,2024-12-31 21:00:00,NaN
26311,-,-,2024-12-31 22:00:00,NaN


### df_all_prices

##### df_prices_volumes

In [10]:
df_prices = pd.read_csv('data/da/day_ahead_prices.csv', index_col=0)
df_prices.rename(columns={'Price': 'price'}, inplace=True)
df_volumes = pd.read_csv('data/da/day_ahead_volumes.csv', index_col=0)
df_volumes.rename(columns={'Price': 'volumes'}, inplace=True)

df_prices_volumes = pd.merge(df_prices, df_volumes, left_index=True, right_index=True, how='left')
df_prices_volumes.drop(columns=['Hour_q_x', 'Hour_q_y'], inplace=True)
df_prices_volumes['datum'] = df_prices_volumes.index
df_prices_volumes.reset_index(drop=True, inplace=True)
df_prices_volumes['datum'] = pd.to_datetime(df_prices_volumes['datum'])
df_prices_volumes['datum'] = df_prices_volumes['datum'].apply(lambda t: t.replace(tzinfo=None))
df_prices_volumes

C:\Users\patri\AppData\Local\Temp\ipykernel_17988\3606153352.py:10: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_prices_volumes['datum'] = pd.to_datetime(df_prices_volumes['datum'])


,price,volumes,datum
0,61.83783,942.200,2022-01-01 00:00:00
1,57.07848,1296.900,2022-01-02 00:00:00
2,20.89311,1130.200,2022-01-03 00:00:00
3,110.14028,1314.300,2022-01-04 00:00:00
4,109.85968,1348.500,2022-01-05 00:00:00
...,...,...,...
25284,116.54000,1563.900,2024-11-14 23:00:00
25285,127.75000,946.800,2024-11-15 23:00:00
25286,128.85000,1356.900,2024-11-16 23:00:00
25287,115.30000,1837.037,2024-11-17 23:00:00


##### df_ida

In [11]:
de_prices = pd.read_csv('data/other/de_prices.csv')
de_prices[['left','right']] = de_prices['MTU (CET/CEST)'].str.split(' - ',expand=True)
de_prices = de_prices[['left', 'Day-ahead (EUR/MWh)']]
de_prices['datum'] = pd.to_datetime(de_prices['left'], format='%d/%m/%Y %H:%M:%S').apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
de_prices.drop(columns='left', inplace=True)
de_prices.head()

,Day-ahead (EUR/MWh),datum
0,0.10,2024-01-01 00:00:00
1,39.91,2024-01-01 00:00:00
2,0.10,2024-01-01 00:15:00
3,-0.04,2024-01-01 00:15:00
4,0.10,2024-01-01 00:30:00


In [12]:
hu_prices = pd.read_csv('data/other/hu_prices.csv')
hu_prices[['left','right']] = hu_prices['MTU (CET/CEST)'].str.split(' - ',expand=True)
hu_prices = hu_prices[['left', 'Day-ahead (EUR/MWh)']]
hu_prices['datum'] = pd.to_datetime(hu_prices['left'], format='%d/%m/%Y %H:%M:%S').apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
hu_prices.drop(columns='left', inplace=True)
hu_prices.tail()

,Day-ahead (EUR/MWh),datum
25267,499.33,2024-11-18 19:00:00
25268,160.24,2024-11-18 20:00:00
25269,127.61,2024-11-18 21:00:00
25270,122.31,2024-11-18 22:00:00
25271,116.61,2024-11-18 23:00:00


In [13]:
# add IDA1, IDA2 and Sipx
ida1 = pd.read_csv('data/ida/ida1_prices.csv', parse_dates=['DeliveryDateTime'])
ida1.rename(columns={'Price': 'IDA1price'}, inplace=True)
ida1 = ida1[ida1['DeliveryDateTime'].isnull() == False]
ida1['DeliveryDateTime'] = ida1['DeliveryDateTime'].apply(lambda t: t.replace(tzinfo=None))

ida2 = pd.read_csv('data/ida/ida2_prices.csv', parse_dates=['DeliveryDateTime'])
ida2.rename(columns={'Price': 'IDA2price'}, inplace=True)
ida2 = ida2[ida2['DeliveryDateTime'].isnull() == False]
ida2['DeliveryDateTime'] = ida2['DeliveryDateTime'].apply(lambda t: t.replace(tzinfo=None))

sipx = pd.read_csv('data/other/sipx_prices.csv')
sipx = sipx[sipx['DeliveryDateTime'] > '2022-01-01']
sipx.rename(columns={'Price': 'SIPXprice'}, inplace=True)
sipx['DeliveryDateTime'] = pd.to_datetime(sipx['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))

# merge all three
idas = pd.merge(ida1, ida2, on='DeliveryDateTime', how='left').fillna(0)
idas['DeliveryDateTime'] = pd.to_datetime(idas['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
df_ida = pd.merge(idas, sipx, on='DeliveryDateTime', how='right')

# clean a bit
df_ida['datum'] = pd.to_datetime(df_ida['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))
df_ida.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0', 'DeliveryDateTime', 'Hour_q_x', 'Hour_q_y'], inplace=True)
df_ida.sort_values(by='datum', inplace=True)
df_ida

C:\Users\patri\AppData\Local\Temp\ipykernel_17988\841790980.py:15: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  sipx['DeliveryDateTime'] = pd.to_datetime(sipx['DeliveryDateTime']).apply(lambda d: d.strftime('%Y-%m-%d %H:%M:%S'))


,IDA1price,IDA2price,SIPXprice,datum
4208,50.10,75.0,61.83783,2022-01-01 00:00:00
4209,NaN,NaN,61.83783,2022-01-01 00:15:00
4210,NaN,NaN,61.83783,2022-01-01 00:30:00
4211,NaN,NaN,61.83783,2022-01-01 00:45:00
8420,50.10,75.0,51.44327,2022-01-01 01:00:00
...,...,...,...,...
92643,102.73,0.0,132.91000,2024-11-18 22:45:00
96852,160.83,0.0,117.72000,2024-11-18 23:00:00
96853,100.63,0.0,117.72000,2024-11-18 23:15:00
96854,83.57,0.0,117.72000,2024-11-18 23:30:00


In [14]:
df_ida = pd.merge(df_ida, hu_prices, on='datum', how='left')
df_ida['datum'] = pd.to_datetime(df_ida['datum'])
df_ida['datum'] = df_ida['datum'].apply(lambda t: t.replace(tzinfo=None))

In [15]:
df_all_prices = pd.merge(df_ida, df_prices_volumes, on='datum', how='left')
df_all_prices['price'] = df_all_prices['price'].ffill()
df_all_prices['volumes'] = df_all_prices['volumes'].ffill()
df_all_prices.sort_values(by='datum', inplace=True)
df_all_prices.tail()

,IDA1price,IDA2price,SIPXprice,datum,Day-ahead (EUR/MWh),price,volumes
101047,102.73,0.0,132.91,2024-11-18 22:45:00,NaN,132.91,1846.592
101048,160.83,0.0,117.72,2024-11-18 23:00:00,116.61,117.72,1950.000
101049,100.63,0.0,117.72,2024-11-18 23:15:00,NaN,117.72,1950.000
101050,83.57,0.0,117.72,2024-11-18 23:30:00,NaN,117.72,1950.000
101051,69.80,0.0,117.72,2024-11-18 23:45:00,NaN,117.72,1950.000


### Merge

In [20]:
df_temp = pd.merge(df_eles_koledar, df_entsoe, on='datum', how='left')
df = pd.merge(df_temp, df_all_prices, on='datum', how='right')
df.tail()

,PREDVIDENA PROIZVODNJA,DEJANSKA PROIZVODNJA,predviden_prevzem,dejanski_prevzem,datum,holiday,temperature_2m,cloud_cover,direct_normal_irradiance,Day-ahead Total Load Forecast [MW] - BZN|SI,Actual Total Load [MW] - BZN|SI,Generation - Solar [MW] Day Ahead/ Slovenia (SI),IDA1price,IDA2price,SIPXprice,Day-ahead (EUR/MWh),price,volumes
101431,NaN,NaN,NaN,NaN,2024-11-18 22:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.73,0.0,132.91,NaN,132.91,1846.592
101432,1015.0,NaN,1416.0,NaN,2024-11-18 23:00:00,NaN,1.627,100.0,0.0,1522,-,NaN,160.83,0.0,117.72,116.61,117.72,1950.000
101433,NaN,NaN,NaN,NaN,2024-11-18 23:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.63,0.0,117.72,NaN,117.72,1950.000
101434,NaN,NaN,NaN,NaN,2024-11-18 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.57,0.0,117.72,NaN,117.72,1950.000
101435,NaN,NaN,NaN,NaN,2024-11-18 23:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.80,0.0,117.72,NaN,117.72,1950.000


In [21]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
	# Drop duplicate rows in column: 'datum'
	df = df.drop_duplicates(subset=['datum'])
	# Renames
	df = df.rename(columns={'datum': 'date', 
							'PREDVIDENA PROIZVODNJA': 'forecast_production',
							'DEJANSKA PROIZVODNJA': 'production',
							'predviden_prevzem': 'forecast_consumption',
							'dejanski_prevzem': 'consumption',
							'Day-ahead Total Load Forecast [MW] - BZN|SI': 'da_load',
							'Actual Total Load [MW] - BZN|SI': 'actual_load',
							'Day-ahead (EUR/MWh)': 'HUprice',
							'Generation - Solar  [MW] Day Ahead/ Slovenia (SI)': 'solar',
							'temperature_2m': 'T',
							'direct_normal_irradiance': 'irradiance',
						})
	df.ffill(inplace=True)
	
	df['date'] = df['date'].astype('datetime64[ns]')
	df = df[df['date'] < '2024-11-19 00:00:00']

	df = df[(df['date'].dt.day != 29) | (df['date'].dt.month != 2)]
	df = df.drop(df.index[0])
	df.drop(columns=['actual_load'], inplace=True)

	return df


df = clean_data(df.copy())
df

,forecast_production,production,forecast_consumption,consumption,date,holiday,T,cloud_cover,irradiance,da_load,solar,IDA1price,IDA2price,SIPXprice,HUprice,price,volumes
1,965.0,968.0,989.0,1044.0,2022-01-01 00:15:00,New Year's Day,2.780,8.0,0.0,1089.0,0.0,50.10,75.0,61.83783,61.84,61.83783,942.200
2,965.0,968.0,989.0,1044.0,2022-01-01 00:30:00,New Year's Day,2.780,8.0,0.0,1089.0,0.0,50.10,75.0,61.83783,61.84,61.83783,942.200
3,965.0,968.0,989.0,1044.0,2022-01-01 00:45:00,New Year's Day,2.780,8.0,0.0,1089.0,0.0,50.10,75.0,61.83783,61.84,61.83783,942.200
4,959.0,949.0,953.0,1018.0,2022-01-01 01:00:00,New Year's Day,3.580,14.0,0.0,1056.0,0.0,50.10,75.0,51.44327,41.33,51.44327,1084.700
5,959.0,949.0,953.0,1018.0,2022-01-01 01:15:00,New Year's Day,3.580,14.0,0.0,1056.0,0.0,50.10,75.0,51.44327,41.33,51.44327,1084.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101431,1160.0,2370.0,1498.0,1647.0,2024-11-18 22:45:00,0,1.527,88.0,0.0,1607,0.0,102.73,0.0,132.91000,122.31,132.91000,1846.592
101432,1015.0,2370.0,1416.0,1647.0,2024-11-18 23:00:00,0,1.627,100.0,0.0,1522,0.0,160.83,0.0,117.72000,116.61,117.72000,1950.000
101433,1015.0,2370.0,1416.0,1647.0,2024-11-18 23:15:00,0,1.627,100.0,0.0,1522,0.0,100.63,0.0,117.72000,116.61,117.72000,1950.000
101434,1015.0,2370.0,1416.0,1647.0,2024-11-18 23:30:00,0,1.627,100.0,0.0,1522,0.0,83.57,0.0,117.72000,116.61,117.72000,1950.000


In [22]:
print(df.columns)

Index(['forecast_production', 'production', 'forecast_consumption',
       'consumption', 'date', 'holiday', 'T', 'cloud_cover', 'irradiance',
       'da_load', 'solar', 'IDA1price', 'IDA2price', 'SIPXprice', 'HUprice',
       'price', 'volumes'],
      dtype='object')


In [23]:
df.to_csv('data/dataset_task2_shortHU.csv', index=False)